# Setup & Define

In [1]:
import requests
import pandas as pd

googleKey = ''

# Get All Data By Year

Note that the range is not inclusive on the higher end so be sure to add 1 year

In [2]:
dfTeams = pd.DataFrame()

mlbYears = list(range(1876, 2019))
mlbUrl = "http://lookup-service-prod.mlb.com/json/named.team_all_season.bam?sport_code='mlb'&all_star_sw='N'&season="

for y in mlbYears:
    r = requests.get(mlbUrl + str(y))
    if r.json()['team_all_season']['queryResults']['totalSize'] == 0:
        continue
    l = r.json()['team_all_season']['queryResults']['row']
    
    dfTemp = pd.DataFrame(data=l)
    dfTeams = dfTeams.append(dfTemp, ignore_index=True)

dfTeams.tail(3)

,active_sw,address,address_city,address_country,address_intl,address_line1,address_line2,address_line3,address_province,address_state,...,team_id,time_zone,time_zone_alt,time_zone_generic,time_zone_num,time_zone_text,venue_id,venue_name,venue_short,website_url
2844,Y,"24 Willie Mays Plaza&#xa;San Francisco, CA&#x9...",San Francisco,,N,24 Willie Mays Plaza,,,,CA,...,137,PT,America/Los_Angeles,PT,-7,PDT,2395,AT&T Park,AT&T Park,sfgiants.com
2845,Y,"700 Clark Street&#xa;St. Louis, MO&#x9;63102",St. Louis,,N,700 Clark Street,,,,MO,...,138,CT,America/Chicago,CT,-5,CDT,2889,Busch Stadium,Busch Stadium,cardinals.com
2846,Y,"1500 South Capitol Street, SE&#xa;Washington, ...",Washington,,N,"1500 South Capitol Street, SE",,,,DC,...,120,ET,America/New_York,ET,-4,EDT,3309,Nationals Park,Nationals Park,nationals.com


# Clean Up Time

In [3]:
dropTargets = ['address','address_city','address_country','address_intl','address_state',\
               'address_line1','address_line2','address_line3',\
               'address_province','address_zip','phone_number',\
               'division','division_full','division_id',\
               'league_abbrev','league_full','league_id','all_star_sw',\
               'sport_code','sport_code_display','sport_code_name','sport_id',\
               'base_url','website_url','store_url',\
               'home_opener','home_opener_time',\
               'time_zone','time_zone_alt','time_zone_generic','time_zone_num','time_zone_text',\
               'file_code','mlb_org_short','mlb_org_brief','bis_team_code',\
               'name','name_short' ,'name_abbrev',\
               'name_display_long','name_display_short','name_display_brief',\
               'venue_name']

df = dfTeams.drop(columns=dropTargets)

df = df.rename(index=str, columns={"active_sw": "active", 'division_abbrev': 'division'})
df.active = df.active.map({'Y': True, 'N': False}).astype('bool')
df.spring_league = df.spring_league_id.map({'':'', '114':'CL', '115':'GL'})

df = df.drop(columns=['spring_league_abbrev','spring_league_full','spring_league_id'])

cols = ['mlb_org_id','mlb_org_abbrev','mlb_org','franchise_code','name_display_full','team_code','season',\
        'city','state','league','division','spring_league',\
        'first_year_of_play','last_year_of_play',\
        'venue_id','venue_short']

df = df[cols]
df = df.sort_values(by=['season'], ascending=False)
df = df.sort_values(by=['mlb_org_abbrev','mlb_org','franchise_code','name_display_full','team_code'])

df.to_csv('teams.csv')
df.to_pickle('teams.pkl')

df.tail(3)

,mlb_org_id,mlb_org_abbrev,mlb_org,franchise_code,name_display_full,team_code,season,city,state,league,division,spring_league,first_year_of_play,last_year_of_play,venue_id,venue_short
2456,120,WSH,Washington Nationals,MON,Washington Nationals,was,2005,Washington,DC,NL,NLE,GL,1969,2019,2721,RFK Stadium
94,219,WSN,Washington Nationals,WS7,Washington Nationals,ws7,1884,,,AA,,,1884,1884,401,
105,222,WST,Washington Nationals,WSU,Washington Nationals,wsu,1884,,,UA,,,1884,1884,401,


# Fill In Some Blanks

In [4]:
df.loc[df['mlb_org'].str.contains('Brooklyn', regex=True), 'city'] = 'New York'
df.loc[df['mlb_org'].str.contains('Brooklyn', regex=True), 'state'] = 'NY'

df.loc[df['mlb_org'].str.contains('Buffalo', regex=True), 'city'] = 'Buffalo'
df.loc[df['mlb_org'].str.contains('Buffalo', regex=True), 'state'] = 'NY'

df.loc[df['mlb_org'].str.contains('Chicago', regex=True), 'city'] = 'Chicago'
df.loc[df['mlb_org'].str.contains('Chicago', regex=True), 'state'] = 'IL'

df.loc[df['mlb_org'].str.contains('Cincinnati', regex=True), 'city'] = 'Cincinnati'
df.loc[df['mlb_org'].str.contains('Cincinnati', regex=True), 'state'] = 'OH'

df.loc[df['mlb_org'].str.contains('Cleveland', regex=True), 'city'] = 'Cleveland'
df.loc[df['mlb_org'].str.contains('Cleveland', regex=True), 'state'] = 'OH'

df.loc[df['mlb_org'].str.contains('Columbus', regex=True), 'city'] = 'Columbus'
df.loc[df['mlb_org'].str.contains('Columbus', regex=True), 'state'] = 'OH'

df.loc[df['mlb_org'].str.contains('Detroit', regex=True), 'city'] = 'Detroit'
df.loc[df['mlb_org'].str.contains('Detroit', regex=True), 'state'] = 'MI'

df.loc[df['mlb_org'].str.contains('Kansas City', regex=True), 'city'] = 'Kansas City'
df.loc[df['mlb_org'].str.contains('Kansas City', regex=True), 'state'] = 'MO'

df.loc[df['mlb_org'].str.contains('Milwaukee', regex=True), 'city'] = 'Milwaukee'
df.loc[df['mlb_org'].str.contains('Milwaukee', regex=True), 'state'] = 'WI'

df.loc[df.name_display_full == 'Indianapolis Hoosier-Feds', 'city'] = 'Indianapolis'
df.loc[df.name_display_full == 'Indianapolis Hoosier-Feds', 'state'] = 'IN'

df.loc[df.name_display_full == 'Newark Peppers', 'city'] = 'Harrison'
df.loc[df.name_display_full == 'Newark Peppers', 'state'] = 'NJ'

df.loc[df['name_display_full'].str.contains('New York', regex=True), 'city'] = 'New York'
df.loc[df['name_display_full'].str.contains('New York', regex=True), 'state'] = 'NY'

df.loc[df.mlb_org == 'Providence Grays', 'city'] = 'Providence'
df.loc[df.mlb_org == 'Providence Grays', 'state'] = 'RI'

df.loc[df.mlb_org == 'Richmond Virginias', 'city'] = 'Richmond'
df.loc[df.mlb_org == 'Richmond Virginias', 'state'] = 'VA'

df.loc[df['mlb_org'].str.contains('St. Louis', regex=True), 'city'] = 'St. Louis'
df.loc[df['mlb_org'].str.contains('St. Louis', regex=True), 'state'] = 'MO'

df.loc[df.mlb_org == 'St. Paul Saints', 'city'] = 'St. Paul'
df.loc[df.mlb_org == 'St. Paul Saints', 'state'] = 'MN'

df.loc[df.mlb_org == 'Syracuse Stars', 'city'] = 'Syracuse'
df.loc[df.mlb_org == 'Syracuse Stars', 'state'] = 'NY'

df.loc[df['mlb_org'].str.contains('Toledo', regex=True), 'city'] = 'Toledo'
df.loc[df['mlb_org'].str.contains('Toledo', regex=True), 'state'] = 'OH'

df.loc[df.name_display_full == 'Washington Nationals', 'city'] = 'Washington'
df.loc[df.name_display_full == 'Washington Nationals', 'state'] = 'DC'

df.loc[df.name_display_full == 'Washington Senators', 'city'] = 'Washington'
df.loc[df.name_display_full == 'Washington Senators', 'state'] = 'DC'

df.loc[df.name_display_full == 'Wilmington Quicksteps', 'city'] = 'Wilmington'
df.loc[df.name_display_full == 'Wilmington Quicksteps', 'state'] = 'DE'

df.to_csv('teams.csv')
df.to_pickle('teams.pkl')

# Geo Code

In [5]:
googleKey = '&key=' + googleKey
googleUrl = 'https://maps.googleapis.com/maps/api/geocode/json?address='

loc = df[['city', 'state']].copy().drop_duplicates().reset_index(drop=True)
loc['cityState'] = loc.city + ', ' + loc.state

df['lat'] = ''
df['lng'] = ''

for index,row in loc.iterrows():
    url = googleUrl + row['cityState'] + googleKey

    r = requests.get(url)
    if r.json()['status'] != 'ZERO_RESULTS':
        lat = r.json()['results'][0]['geometry']['location']['lat']
        lng = r.json()['results'][0]['geometry']['location']['lng']
    
        df.loc[(df.city == row['city']) & (df.state == row['state']), 'lat'] = lat
        df.loc[(df.city == row['city']) & (df.state == row['state']), 'lng'] = lng

df.lat = pd.to_numeric(df.lat, errors="coerce")
df.lng = pd.to_numeric(df.lng, errors="coerce")

df.reset_index(drop=True)

df.to_csv('teams.csv')
df.to_pickle('teams.pkl')

df.tail(3)

,mlb_org_id,mlb_org_abbrev,mlb_org,franchise_code,name_display_full,team_code,season,city,state,league,division,spring_league,first_year_of_play,last_year_of_play,venue_id,venue_short,lat,lng
2456,120,WSH,Washington Nationals,MON,Washington Nationals,was,2005,Washington,DC,NL,NLE,GL,1969,2019,2721,RFK Stadium,38.907192,-77.036871
94,219,WSN,Washington Nationals,WS7,Washington Nationals,ws7,1884,Washington,DC,AA,,,1884,1884,401,,38.907192,-77.036871
105,222,WST,Washington Nationals,WSU,Washington Nationals,wsu,1884,Washington,DC,UA,,,1884,1884,401,,38.907192,-77.036871


# Done

All Done